In [1]:
from pyspark.sql.functions import desc, explode
import pandas as pd
import datetime
import json
import itertools
import pickle

In [2]:
jsondf = spark.read.json("s3a://groupsinfonewsplitter/2017/*/*/*")

In [3]:
# jsondf.printSchema()

root
 |-- category: struct (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- shortname: string (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- created: long (nullable = true)
 |-- description: string (nullable = true)
 |-- group_photo: struct (nullable = true)
 |    |-- base_url: string (nullable = true)
 |    |-- highres_link: string (nullable = true)
 |    |-- photo_id: long (nullable = true)
 |    |-- photo_link: string (nullable = true)
 |    |-- thumb_link: string (nullable = true)
 |    |-- type: string (nullable = true)
 |-- id: long (nullable = true)
 |-- join_mode: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- link: string (nullable = true)
 |-- lon: double (nullable = true)
 |-- members: long (nullable = true)
 |-- name: string (nullable = true)
 |-- organizer: struct (nullable = true)
 |    |-- member_id: long (nullable = true)
 |    |-- name: string (

In [3]:
selected_features_df = jsondf.select(jsondf['id'],\
                                     jsondf['city'],\
                                     jsondf['category'].alias("cat"),\
                                     jsondf['topics'].alias("top"))

In [4]:
df = selected_features_df.withColumn("category", selected_features_df['cat'].getField("name"))
df = df.withColumn("topics", df['top'].getField("name"))

In [5]:
df.show(5)

+-------+--------+--------------------+--------------------+-----------------+--------------------+
|     id|    city|                 cat|                 top|         category|              topics|
+-------+--------+--------------------+--------------------+-----------------+--------------------+
|6277092| Houston|[31,socializing,s...|[[2494,Black Prof...|      socializing|[Black Profession...|
| 674822| Atlanta|[1,fine arts/cult...|[[542,Arthouse,ar...|fine arts/culture|[Arthouse, Watchi...|
|1104060|  Denver|      [34,tech,tech]|[[455,PHP,php], [...|             tech|[PHP, Open Source...|
|1153075|San Jose|[3,cars/motorcycl...|[[498,Singles,sin...| cars/motorcycles|[Singles, Honda M...|
| 503295| Seattle|      [34,tech,tech]|[[1081,Graphical ...|             tech|[Graphical User I...|
+-------+--------+--------------------+--------------------+-----------------+--------------------+
only showing top 5 rows



In [6]:
df = df.select(df['id'],df['city'],df['category'],df['topics']).distinct()

In [7]:
df.show(5)

+--------+-------------+--------------------+--------------------+
|      id|         city|            category|              topics|
+--------+-------------+--------------------+--------------------+
|  219419|      Seattle|language/ethnic i...|[Chinese Culture,...|
|  219991|     New York|             support|[Social Networkin...|
|  349667|      Seattle|    health/wellbeing|[Yoga, Metaphysic...|
|12612322|      Houston|  movements/politics|[Feminism, Procho...|
|  366695|Mountain View|  outdoors/adventure|[Fitness, Camping...|
+--------+-------------+--------------------+--------------------+
only showing top 5 rows



In [8]:
df.write.parquet("s3a://meetupdf/topic_by_category")